<a href="https://colab.research.google.com/github/kuroro-31/stock-ai-api/blob/master/LSTM%E6%A0%AA%E4%BE%A1%E4%BA%88%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from kerastuner.tuners import RandomSearch
from datetime import datetime

In [ ]:
# テクニカル指標を計算する関数
def calculate_technical_indicators(data):
    data['EMA12'] = data['Close'].ewm(span=12).mean()
    data['EMA26'] = data['Close'].ewm(span=26).mean()
    data['MACD'] = data['EMA12'] - data['EMA26']
    data['Signal'] = data['MACD'].ewm(span=9).mean()
    data['RSI'] = data['Close'].rolling(window=14).apply(lambda x: 100 - (100 / (1 + (x.diff().dropna().apply(lambda x: x if x >= 0 else 0).sum() / x.diff().dropna().apply(lambda x: x if x < 0 else 0).abs().sum()))))

# データ取得
symbol = "AAPL"
start_date = "2020-01-01"
end_date = "2023-05-01"

data = yf.download(symbol, start=start_date, end=end_date)
data = data[['Close']]

calculate_technical_indicators(data)

In [82]:
# ファンダメンタル指標の取得（P/E比）
pe_ratio = yf.Ticker(symbol).info['trailingPE']
data['PE'] = pe_ratio

# 欠損値の削除
data = data.dropna()

In [83]:
# データの前処理
look_back = 60
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

train_size = int(len(data_scaled) * 0.8)
train, test = data_scaled[0:train_size], data_scaled[train_size - look_back:]

def create_dataset(data, look_back):
    X, Y = [], []
    for i in range(len(data) - look_back - 1):
        X.append(data[i:(i + look_back), :])
        Y.append(data[i + look_back, 0])
    return np.array(X), np.array(Y)

X_train, y_train = create_dataset(train, look_back)
X_test, y_test = create_dataset(test, look_back)

In [84]:
# LSTMモデルの構築(LSTM層が4つ)
# Keras Tunerを使用してハイパーパラメータをチューニング
# ドロップアウト率を0.3に
def build_lstm_model(hp):
    model = Sequential()
    model.add(LSTM(hp.Int("input_units", min_value=30, max_value=150, step=30), return_sequences=True, input_shape=(look_back, data.shape[1])))
    model.add(Dropout(hp.Float("input_dropout", min_value=0.1, max_value=0.5, step=0.1)))
    model.add(LSTM(hp.Int("layer_1_units", min_value=30, max_value=150, step=30), return_sequences=True))
    model.add(Dropout(hp.Float("layer_1_dropout", min_value=0.1, max_value=0.5, step=0.1)))
    model.add(LSTM(hp.Int("layer_2_units", min_value=30, max_value=150, step=30), return_sequences=True))
    model.add(Dropout(hp.Float("layer_2_dropout", min_value=0.1, max_value=0.5, step=0.1)))
    model.add(LSTM(hp.Int("layer_3_units", min_value=30, max_value=150, step=30)))
    model.add(Dropout(hp.Float("layer_3_dropout", min_value=0.1, max_value=0.5, step=0.1)))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [ ]:
# ハイパーパラメータのチューニング
tuner = RandomSearch(
    build_lstm_model,
    objective="val_loss",
    max_trials=20,
    executions_per_trial=2,
    directory="random_search",
    project_name="stock_prediction"
)

tuner.search_space_summary()

In [86]:
tuner.search(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=0)

In [87]:
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]

In [ ]:
# 予測
train_predict = best_model.predict(X_train)
test_predict = best_model.predict(X_test)

train_predict = scaler.inverse_transform(np.hstack((train_predict, np.zeros((train_predict.shape[0], data.shape[1]-1)))))
test_predict = scaler.inverse_transform(np.hstack((test_predict, np.zeros((test_predict.shape[0], data.shape[1]-1)))))

print("Train RMSE:", np.sqrt(mean_squared_error(y_train, train_predict[:, 0])))
print("Test RMSE:", np.sqrt(mean_squared_error(y_test, test_predict[:, 0])))

In [ ]:
# 予測結果のプロット
import matplotlib.pyplot as plt

plt.plot(data.index, data['Close'], label="Actual")

# Trainデータの予測結果をプロット
train_range = data.index[look_back:len(train_predict) + look_back]
plt.plot(train_range, train_predict[:, 0], label="Train Prediction")

# Testデータの予測結果をプロット
test_range = data.index[len(train_predict) + look_back + 1:len(train_predict) + len(test_predict) + look_back + 1]
plt.plot(test_range, test_predict, label="Test Prediction")


plt.legend()
plt.show()